# mnist_cnn_deep
### MNIST and Convolutional Neural Network
### L1,L2 : conv2d + relu + max_pool 
### L3 : FC(Fully Connected Layer)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)

In [2]:
# mnist 데이터 가져오기
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# one-hot 인코딩
nb_classes = 10 # 분류 class의 갯수(0~9)

Y_one_hot = tf.one_hot(y_train, nb_classes)  #(60000,)
Y_one_hot.shape                              #(60000,10)

TensorShape([60000, 10])

In [4]:
x_train = tf.cast(x_train,dtype=tf.float32)
x_test = tf.cast(x_test,dtype=tf.float32)

In [5]:
# X값의 shape을 4차원으로 변환                     # 무조건 4차원  아브렐슈드3관 마름모
X_img = tf.reshape(x_train,[-1,28,28,1])
print(X_img.shape)                         # (60000, 28, 28, 1)

(60000, 28, 28, 1)


In [6]:
# Layer 1 : conv2d - relu - max_pool
# (?, 28, 28, 1) --> (?, 14, 14, 32)
 
# <1> conv2d
# L1 input image shape : (?, 28, 28, 1)
# filter : (3,3,1,32), 필터 32개
# strides : (1,1,1,1), padding='SAME'    출력도 같은 크기로 나옴 
# 출력 이미지 : (28+2 - 3)/1 + 1 = 28
# (?, 28, 28, 1) --> (?, 28, 28, 32)

W1 = tf.Variable(tf.random.normal([3,3,1,32]),name='weight1')

def L1_conv2d(X):
    return tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding = 'SAME')

# <2> relu
def L1_relu(X):
    return tf.nn.relu(L1_conv2d(X))     #shape 변화가 없다

# <3> max_pool
# input image : (?, 28, 28, 32)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (28+1 - 2)/2 + 1 = 14
#  (?, 28, 28, 32) -->  (?, 14, 14, 32)
def L1_MaxPool(X):
    return tf.nn.max_pool(L1_relu(X), ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # ksize 는 커널사이즈

In [7]:
# Layer 2 : conv2d - relu - max_pool
# (?, 14, 14, 32) --> (?, 7, 7, 64)
 
# <1> conv2d
# L1 input image shape : (?, 14, 14, 32)
# filter : (3,3,32,64), 필터 32개
# strides : (1,1,1,1), padding='SAME'    출력도 같은 크기로 나옴 
# 출력 이미지 : (14+2 - 3)/1 + 1 = 14
# (?, 14, 14, 32) --> (?, 14, 14, 64)

W2 = tf.Variable(tf.random.normal([3,3,32,64]),name='weight2')

def L2_conv2d(X):
    return tf.nn.conv2d(L1_MaxPool(X),W2,strides=[1,1,1,1],padding = 'SAME')

# <2> relu
def L2_relu(X):
    return tf.nn.relu(L2_conv2d(X))     #shape 변화가 없다

# <3> max_pool
# input image : (?, 14, 14, 64)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (14+1 - 2)/2 + 1 = 7
#  (?, 14, 14, 64) -->  (?, 7, 7, 64)
def L2_MaxPool(X):
    return tf.nn.max_pool(L2_relu(X), ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # ksize 는 커널사이즈

# <4> flatten layer : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L2_flat(X):
    return tf.reshape(L2_MaxPool(X), [-1,7*7*64])

In [8]:
# Layer 3 : FC(Fully Connected Layer)
# (?,7*7*64) * (7*7*64, 10) = (?,10)
nb_classes = 10
W3 = tf.Variable(tf.random.normal([7*7*64,nb_classes]),name='weight3')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')

In [9]:
# 예측 함수(hypothesis) : H(X) = softmax(W*X + b)
def logits(X):
    return tf.matmul(L2_flat(X),W3) + b

def hypothesis(X):
    return tf.nn.softmax(logits(X))

In [10]:
# batch 사이즈로 나누어 학습, 효율적 이며 학습 시간 단축
training_epoch = 50
batch_size = 600

#경사 하강법
# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.optimizers.Adam(learning_rate=0.01)

#학습 시작
print('***** Start Learning!!')
for epoch in range(training_epoch):  #  50회
    avg_cost = 0
    total_batch = int(x_train.shape[0] / batch_size)   # 100 = 60000 / 600 (전체데이터를 batch_size로 나눈 횟수)
    for k in range(total_batch) :
        batch_xs = x_train[k * batch_size : k*batch_size + batch_size] # 600개의 x 데이터
        batch_ys = Y_one_hot[k * batch_size : k*batch_size + batch_size] # 600개의 Y 데이터

        # x값의 shape을 4차원으로 변환
        X_img = tf.reshape(batch_xs,[-1,28,28,1])  
        # 비용함수 구현 방법 : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
        def cost_func_batch():
            cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits(X_img),
                                                             labels = batch_ys)
            cost = tf.reduce_mean(cost_i)
            return cost

        optimizer.minimize(cost_func_batch,var_list=[W1,W2,W3,b])
        avg_cost += cost_func_batch().numpy() / total_batch   # 평균 비용 (개수로 나눈 걸 다 더함)

    print('Epoch:','%04d'%(epoch + 1),'cost:','{:.9f}'.format(avg_cost))
print('***** Learning Finished!!')


***** Start Learning!!
Epoch: 0001 cost: 16350.877973633
Epoch: 0002 cost: 2602.741390381
Epoch: 0003 cost: 1498.446667786
Epoch: 0004 cost: 1014.855358276
Epoch: 0005 cost: 754.467217255
Epoch: 0006 cost: 579.923805923
Epoch: 0007 cost: 456.708410187
Epoch: 0008 cost: 370.457182026
Epoch: 0009 cost: 315.862138405
Epoch: 0010 cost: 256.596597557
Epoch: 0011 cost: 219.776536026
Epoch: 0012 cost: 181.298711648
Epoch: 0013 cost: 157.049478645
Epoch: 0014 cost: 127.052940435
Epoch: 0015 cost: 94.352319641
Epoch: 0016 cost: 72.297872360
Epoch: 0017 cost: 69.243633177
Epoch: 0018 cost: 59.629100535
Epoch: 0019 cost: 54.233009040
Epoch: 0020 cost: 44.924261066
Epoch: 0021 cost: 36.190608300
Epoch: 0022 cost: 31.053305846
Epoch: 0023 cost: 27.441334740
Epoch: 0024 cost: 27.921191647
Epoch: 0025 cost: 22.072800317
Epoch: 0026 cost: 16.191557769
Epoch: 0027 cost: 13.946444909
Epoch: 0028 cost: 15.119055512
Epoch: 0029 cost: 17.530641075
Epoch: 0030 cost: 12.917535325
Epoch: 0031 cost: 8.12101600

In [11]:
# 정확도 측정 : accuracy computation

# y_test 값의 one-hot 인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)    # (10000,10)
# print(Y_one_hot.shape)                       # (10000,10)  , (2차원)

# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)

# X값의 shape을 4차원으로 변환
X_img = tf.reshape(x_test,[-1,28,28,1])
        
correct_predict = tf.equal(predict(X_img),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))
print("Accuracy:",accuracy.numpy()) # Accuracy: 0.9534

#예측
print('***** Predict')
pred = predict(X_img).numpy()
print(pred,y_test)


Accuracy: 0.9815
***** Predict
[7 2 1 ... 4 5 6] [7 2 1 ... 4 5 6]
